In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import geohash
import random
import math
import pytz
import time

In [246]:
# raw_data = {'first_name': ['Jason', 'Molly', 'Tina', 'Jake', 'Amy'],
#         'last_name': ['Miller', 'Jacobson', ".", 'Milner', 'Cooze'],
#         'age': [42, 52, 36, 24, 73],
#         'preTestScore': [4, 24, 31, ".", "."],
#         'postTestScore': ["25,000", "94,000", 57, 62, 70]}
# df = pd.DataFrame(raw_data, columns = ['first_name', 'last_name', 'age', 'preTestScore', 'postTestScore'])
# df
inputdf = pd.read_csv("train.csv")
df = inputdf

In [247]:
df[:4]

,row_id,start_lng,start_lat,end_lng,end_lat,start_timestamp,duration
0,0,-74.009087,40.713818,-74.004326,40.719986,1420950819,112
1,1,-73.971176,40.762428,-74.004181,40.742653,1420950819,1159
2,2,-73.994957,40.745079,-73.999939,40.734650,1421377541,281
3,3,-73.991127,40.750080,-73.988609,40.734890,1421377542,636


In [249]:
def geohash_encode_start(x):
    if x['start_lat'] == 90.0:
        return 'z' * precision
    precision = 7
    if (x['start_lat'] < -1000.0 or x['start_lat'] > 1000.0 or x['start_lng'] < -1000.0 or x['start_lng'] > 1000.0):
        return 0
    return geohash.encode(x['start_lat'], x['start_lng'], precision)

def geohash_encode_end(x):
    if x['end_lat'] == 90.0:
        return 'z' * precision
    precision = 7
    if (x['start_lat'] < -1000.0 or x['start_lat'] > 1000.0 or x['start_lng'] < -1000.0 or x['start_lng'] > 1000.0):
        return 0    
    return geohash.encode(x['end_lat'], x['end_lng'], precision)


# print 'start geohash'
# df['start_geohash'] = df.apply(geohash_encode_start, axis=1)
# print 'end geohash'
# df['end_geohash'] = df.apply(geohash_encode_end, axis=1)
    
# df["start_geohash"] = geohash_encode(df["start_lng"], df[""])

In [250]:
print 'start geohash'
df['start_geohash'] = df.apply(geohash_encode_start, axis=1)
print 'end geohash'
df['end_geohash'] = df.apply(geohash_encode_end, axis=1)


start geohash
end geohash


In [262]:
from datetime import datetime
test_time = df[:1]["start_timestamp"]
tz = pytz.timezone('America/New_York')
def get_local_time(x):
    d = datetime.utcfromtimestamp(x["start_timestamp"]).replace(tzinfo=pytz.utc)
    dt = d.astimezone(tz)
    return dt

df["start_timestamp_obj"] = df.apply(get_local_time, axis=1)
df[:4]
# dt = utc_dt.astimezone(tz)
# dt = utc_dt.astimezone(tz)
# print(dt.strftime('%Y-%m-%d %H:%M:%S %Z%z'))

,row_id,start_lng,start_lat,end_lng,end_lat,start_timestamp,duration,start_geohash,end_geohash,start_timestamp_obj
0,0,-74.009087,40.713818,-74.004326,40.719986,1420950819,112,dr5regs,dr5reuz,2015-01-10 23:33:39-05:00
1,1,-73.971176,40.762428,-74.004181,40.742653,1420950819,1159,dr5rusx,dr5rgbz,2015-01-10 23:33:39-05:00
2,2,-73.994957,40.745079,-73.999939,40.734650,1421377541,281,dr5ru1q,dr5rspd,2015-01-15 22:05:41-05:00
3,3,-73.991127,40.750080,-73.988609,40.734890,1421377542,636,dr5ru63,dr5rsre,2015-01-15 22:05:42-05:00


In [263]:
def get_start_of_the_day(x):
    return x['start_timestamp_obj'].replace(hour=0, minute=0, second=0)
df["start_of_day"] = df.apply(get_start_of_the_day, axis=1)
df[:4]

,row_id,start_lng,start_lat,end_lng,end_lat,start_timestamp,duration,start_geohash,end_geohash,start_timestamp_obj,start_of_day
0,0,-74.009087,40.713818,-74.004326,40.719986,1420950819,112,dr5regs,dr5reuz,2015-01-10 23:33:39-05:00,2015-01-10 00:00:00-05:00
1,1,-73.971176,40.762428,-74.004181,40.742653,1420950819,1159,dr5rusx,dr5rgbz,2015-01-10 23:33:39-05:00,2015-01-10 00:00:00-05:00
2,2,-73.994957,40.745079,-73.999939,40.734650,1421377541,281,dr5ru1q,dr5rspd,2015-01-15 22:05:41-05:00,2015-01-15 00:00:00-05:00
3,3,-73.991127,40.750080,-73.988609,40.734890,1421377542,636,dr5ru63,dr5rsre,2015-01-15 22:05:42-05:00,2015-01-15 00:00:00-05:00


In [297]:
import pytz
from datetime import datetime
test_time = df[:1]["start_timestamp"]
tz = pytz.timezone('America/New_York')
def get_local_time(timestamp):
    d = datetime.utcfromtimestamp(timestamp).replace(tzinfo=pytz.utc)
    dt = d.astimezone(tz)
    return dt
d = df[:1000].copy()
start_time = d["start_timestamp"].apply(get_local_time)
# start_time[:10]
# d[['start_timestamp']]
# start_time
start_time.name = 'start_timestamp_obj1'
pd.concat([d, pd.DataFrame(start_time, index=d.index)], axis = 1)
# d[:10]
# df["start_timestamp_obj"] = df.apply(get_local_time, axis=1)
# df[:4]
# dt = utc_dt.astimezone(tz)
# dt = utc_dt.astimezone(tz)
# print(dt.strftime('%Y-%m-%d %H:%M:%S %Z%z'))

,row_id,start_lng,start_lat,end_lng,end_lat,start_timestamp,duration,start_geohash,end_geohash,start_timestamp_obj,start_of_day,weekday,start_timestamp_obj1
0,0,-74.009087,40.713818,-74.004326,40.719986,1420950819,112,dr5regs,dr5reuz,2015-01-10 23:33:39-05:00,2015-01-10 00:00:00-05:00,5,2015-01-10 23:33:39-05:00
1,1,-73.971176,40.762428,-74.004181,40.742653,1420950819,1159,dr5rusx,dr5rgbz,2015-01-10 23:33:39-05:00,2015-01-10 00:00:00-05:00,5,2015-01-10 23:33:39-05:00
2,2,-73.994957,40.745079,-73.999939,40.734650,1421377541,281,dr5ru1q,dr5rspd,2015-01-15 22:05:41-05:00,2015-01-15 00:00:00-05:00,3,2015-01-15 22:05:41-05:00
3,3,-73.991127,40.750080,-73.988609,40.734890,1421377542,636,dr5ru63,dr5rsre,2015-01-15 22:05:42-05:00,2015-01-15 00:00:00-05:00,3,2015-01-15 22:05:42-05:00
4,4,-73.945511,40.773724,-73.987434,40.755707,1422173586,705,dr5rvqd,dr5ru77,2015-01-25 03:13:06-05:00,2015-01-25 00:00:00-05:00,6,2015-01-25 03:13:06-05:00
5,5,-73.992828,40.701485,-73.992905,40.719978,1422173589,509,dr5rs32,dr5rskb,2015-01-25 03:13:09-05:00,2015-01-25 00:00:00-05:00,6,2015-01-25 03:13:09-05:00
6,6,-73.955688,40.772469,-73.974380,40.756672,1422173589,441,dr5rvn7,dr5ruet,2015-01-25 03:13:09-05:00,2015-01-25 00:00:00-05:00,6,2015-01-25 03:13:09-05:00
7,7,-73.991249,40.765461,-73.987534,40.768410,1420407892,106,dr5rum1,dr5rume,2015-01-04 16:44:52-05:00,2015-01-04 00:00:00-05:00,6,2015-01-04 16:44:52-05:00
8,8,-73.870819,40.773701,-73.968697,40.762680,1421359241,1861,dr5rzn8,dr5ruu9,2015-01-15 17:00:41-05:00,2015-01-15 00:00:00-05:00,3,2015-01-15 17:00:41-05:00
9,9,-73.971817,40.756901,-73.974548,40.751736,1422304871,210,dr5ruex,dr5rudt,2015-01-26 15:41:11-05:00,2015-01-26 00:00:00-05:00,0,2015-01-26 15:41:11-05:00


In [298]:
# df.drop('column_name', axis=1, inplace=True)
df[:2]

,row_id,start_lng,start_lat,end_lng,end_lat,start_timestamp,duration,start_geohash,end_geohash,start_timestamp_obj,start_of_day,weekday
0,0,-74.009087,40.713818,-74.004326,40.719986,1420950819,112,dr5regs,dr5reuz,2015-01-10 23:33:39-05:00,2015-01-10 00:00:00-05:00,5
1,1,-73.971176,40.762428,-74.004181,40.742653,1420950819,1159,dr5rusx,dr5rgbz,2015-01-10 23:33:39-05:00,2015-01-10 00:00:00-05:00,5


In [266]:
tz = pytz.timezone('America/New_York')
epoch = datetime.utcfromtimestamp(0).replace(tzinfo=pytz.utc)
epoch = epoch.astimezone(tz)
temp = df.iloc[1]["start_of_day"]
temputc = temp.astimezone(tz)
print temp.astimezone(tz)
print (temp - epoch).total_seconds()
# print datetime.utcfromtimestamp(temputc)
# print 

# print datetime.utcfromtimestamp(dff.iloc[1]["start_of_day"])

2015-01-10 00:00:00-05:00
1420866000.0


In [279]:
def get_hour(x):
    return x["start_timestamp_obj"].hour
def get_min(x):
    return x["start_timestamp_obj"].minute
def get_weekday(x):
    return x["start_timestamp_obj"].weekday()

def get_all_times(x):
    return (x["start_timestamp_obj"].hour, x["start_timestamp_obj"].minute, x["start_timestamp_obj"].weekday())

d = df[:20]
d
hour = d["start_timestamp_obj"].dt.hour
hour.name = 'hour'
pd.concat([d, hour], axis = 1)
# df["weekday"] = df.apply(get_weekday, axis=1)
# df["hour"] = df.apply(get_hour, axis=1)
# df["minute"] = df.apply(get_min, axis=1)
# d["hour", "minute", "weekday"] = d.apply(get_all_times, axis=1)
# d["hour"] = d["start_timestamp_obj"].hour


,row_id,start_lng,start_lat,end_lng,end_lat,start_timestamp,duration,start_geohash,end_geohash,start_timestamp_obj,start_of_day,weekday,hour
0,0,-74.009087,40.713818,-74.004326,40.719986,1420950819,112,dr5regs,dr5reuz,2015-01-10 23:33:39-05:00,2015-01-10 00:00:00-05:00,5,23
1,1,-73.971176,40.762428,-74.004181,40.742653,1420950819,1159,dr5rusx,dr5rgbz,2015-01-10 23:33:39-05:00,2015-01-10 00:00:00-05:00,5,23
2,2,-73.994957,40.745079,-73.999939,40.734650,1421377541,281,dr5ru1q,dr5rspd,2015-01-15 22:05:41-05:00,2015-01-15 00:00:00-05:00,3,22
3,3,-73.991127,40.750080,-73.988609,40.734890,1421377542,636,dr5ru63,dr5rsre,2015-01-15 22:05:42-05:00,2015-01-15 00:00:00-05:00,3,22
4,4,-73.945511,40.773724,-73.987434,40.755707,1422173586,705,dr5rvqd,dr5ru77,2015-01-25 03:13:06-05:00,2015-01-25 00:00:00-05:00,6,3
5,5,-73.992828,40.701485,-73.992905,40.719978,1422173589,509,dr5rs32,dr5rskb,2015-01-25 03:13:09-05:00,2015-01-25 00:00:00-05:00,6,3
6,6,-73.955688,40.772469,-73.974380,40.756672,1422173589,441,dr5rvn7,dr5ruet,2015-01-25 03:13:09-05:00,2015-01-25 00:00:00-05:00,6,3
7,7,-73.991249,40.765461,-73.987534,40.768410,1420407892,106,dr5rum1,dr5rume,2015-01-04 16:44:52-05:00,2015-01-04 00:00:00-05:00,6,16
8,8,-73.870819,40.773701,-73.968697,40.762680,1421359241,1861,dr5rzn8,dr5ruu9,2015-01-15 17:00:41-05:00,2015-01-15 00:00:00-05:00,3,17
9,9,-73.971817,40.756901,-73.974548,40.751736,1422304871,210,dr5ruex,dr5rudt,2015-01-26 15:41:11-05:00,2015-01-26 00:00:00-05:00,0,15


In [ ]:
df.to_csv("train_mod.csv", index=False)

In [12]:
from collections import defaultdict
g = set()
dictionary = {}
reverse_dictionary = {}
for index, row in df.iterrows():
    g.add(row['start_geohash'])
    g.add(row['end_geohash'])
for i, item in enumerate(g):
    dictionary[i] = item
    reverse_dictionary[item] = i

tup = defaultdict(list)
for index, row in df.iterrows():
    tup[row['start_geohash']].append((row['duration'], row['end_geohash']))
glist = list(g)
    
#     gmap[row['start_geohash']]

In [13]:
sorted_tup = {}
for k, _ in tup.iteritems():
    sorted_tup[k] = sorted(tup[k])
del tup

In [14]:
data_index = 0


In [15]:
def generate_batch(batch_size):
    global data_index
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    dictionary[data_index]
    i = 0
    while i < batch_size:
        item = data_index
        element = sorted_tup.get(dictionary[data_index])
        if not element:
            data_index = (data_index + 1) % len(sorted_tup)
            continue
        options = sorted_tup[dictionary[data_index]]
        choice = random.choice(options[:3])
        batch[i] = item
        labels[i, 0] = reverse_dictionary[choice[1]]
        data_index = (data_index + 1) % len(sorted_tup)
        i += 1
    return batch, labels


In [23]:
batch_size = 64
embedding_size = 64  # Dimension of the embedding vector.
num_sampled = 64
vocabulary_size = len(g)

graph = tf.Graph()

with graph.as_default():

  # Input data.
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])

    # Ops and variables pinned to the CPU because of missing GPU implementation
    with tf.device('/cpu:0'):
    # Look up embeddings for inputs.
        embeddings = tf.Variable(
            tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)

        # Construct the variables for the NCE loss
        nce_weights = tf.Variable(
            tf.truncated_normal([vocabulary_size, embedding_size],
                                stddev=1.0 / math.sqrt(embedding_size)))
        nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

  # Compute the average NCE loss for the batch.
  # tf.nce_loss automatically draws a new sample of the negative labels each
  # time we evaluate the loss.
    loss = tf.reduce_mean(
      tf.nn.nce_loss(weights=nce_weights,
                     biases=nce_biases,
                     labels=train_labels,
                     inputs=embed,
                     num_sampled=num_sampled,
                     num_classes=vocabulary_size))

    # Construct the SGD optimizer using a learning rate of 1.0.
    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

    # Compute the cosine similarity between minibatch examples and all embeddings.
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
#     valid_embeddings = tf.nn.embedding_lookup(
#       normalized_embeddings, valid_dataset)
#     similarity = tf.matmul(
#       valid_embeddings, normalized_embeddings, transpose_b=True)
    init = tf.global_variables_initializer()


In [25]:
# Step 5: Begin training.
num_steps = 100000
with tf.Session(graph=graph) as session:
    # We must initialize all variables before we use them.
    init.run()
    print('Initialized')

    average_loss = 0
    for step in xrange(num_steps):
        batch_inputs, batch_labels = generate_batch(
            batch_size)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

        # We perform one update step by evaluating the optimizer op (including it
        # in the list of returned values for session.run()
        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val

        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000
            # The average loss is an estimate of the loss over the last 2000 batches.
            print('Average loss at step ', step, ': ', average_loss)
            average_loss = 0

        # Note that this is expensive (~20% slowdown if computed every 500 steps)
#         if step % 10000 == 0:
#             sim = similarity.eval()
#             for i in xrange(valid_size):
#                 valid_word = reverse_dictionary[valid_examples[i]]
#                 top_k = 8  # number of nearest neighbors
#                 nearest = (-sim[i, :]).argsort()[1:top_k + 1]
#                 log_str = 'Nearest to %s:' % valid_word
#                 for k in xrange(top_k):
#                     close_word = reverse_dictionary[nearest[k]]
#                     log_str = '%s %s,' % (log_str, close_word)
#                 print(log_str)
    final_embeddings = normalized_embeddings.eval()


Initialized
('Average loss at step ', 0, ': ', 211.41249084472656)
('Average loss at step ', 2000, ': ', 27.474664728134872)
('Average loss at step ', 4000, ': ', 0.72894951388239859)
('Average loss at step ', 6000, ': ', 0.30536760621517894)
('Average loss at step ', 8000, ': ', 0.22277428605407476)
('Average loss at step ', 10000, ': ', 0.18663396403193475)
('Average loss at step ', 12000, ': ', 0.16558996750600635)
('Average loss at step ', 14000, ': ', 0.15344791367277502)
('Average loss at step ', 16000, ': ', 0.14318301725946367)
('Average loss at step ', 18000, ': ', 0.13272839654795826)
('Average loss at step ', 20000, ': ', 0.12843330964446067)
('Average loss at step ', 22000, ': ', 0.12636266222503037)
('Average loss at step ', 24000, ': ', 0.12250038140360266)
('Average loss at step ', 26000, ': ', 0.12035442180838436)
('Average loss at step ', 28000, ': ', 0.11792017829511314)
('Average loss at step ', 30000, ': ', 0.11716862607561052)
('Average loss at step ', 32000, ': ',

In [ ]:
final_embeddings[:1]

In [240]:
input_batch = 32
data_size = 50000
training_size = 40000
testing_size = data_size - training_size
INPUT_FEATURES = 131
# start_timestamp	duration	start_geohash	end_geohash	start_timestamp_obj	start_of_day	weekday	hour	minute
""" Generate the input feature dataset """
def get_batch(batch_size, df):
    global batch_index
    batch = np.ndarray(shape=(batch_size, INPUT_FEATURES), dtype=np.float32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.float32)
    for i in range(batch_size):
        index = (batch_index + i) % training_size
        batch[i, 0] = df.iloc[index]["weekday"]
        batch[i, 1] = df.iloc[index]["hour"]
        batch[i, 2] = df.iloc[index]["minute"]
        start_geohash_embedding = final_embeddings[reverse_dictionary[df.iloc[index]["start_geohash"]]]
        end_geohash_embedding = final_embeddings[reverse_dictionary[df.iloc[index]["end_geohash"]]]
        embedding_start_index = 3
        for j in range(embedding_size):
            batch[i, j + embedding_start_index] = start_geohash_embedding[j]
        end_geohash_emebedding_start = embedding_start_index + embedding_size
        for j in range(embedding_size):
            batch[i, j + end_geohash_emebedding_start] = start_geohash_embedding[j]
        labels[i, 0] = df.iloc[index]["duration"]
    batch_index += batch_size % training_size
    return batch, labels

def get_test_batch(batch_size, df):
    indices = []
    for i in range(batch_size):
        indices.append(random.randrange(0, len(df)))
    batch = np.ndarray(shape=(batch_size, INPUT_FEATURES), dtype=np.float32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.float32)
    for i, position in enumerate(indices):
        batch[i, 0] = df.iloc[position]["weekday"]
        batch[i, 1] = df.iloc[position]["hour"]
        batch[i, 2] = df.iloc[position]["minute"]
        start_geohash_embedding = final_embeddings[reverse_dictionary[df.iloc[position]["start_geohash"]]]
        end_geohash_embedding = final_embeddings[reverse_dictionary[df.iloc[position]["end_geohash"]]]
        embedding_start_index = 3
        for j in range(embedding_size):
            batch[i, j + embedding_start_index] = start_geohash_embedding[j]
        end_geohash_emebedding_start = embedding_start_index + embedding_size
        for j in range(embedding_size):
            batch[i, j + end_geohash_emebedding_start] = start_geohash_embedding[j]
        labels[i, 0] = df.iloc[position]["duration"]
    return batch, labels


In [237]:
c = get_test_batch(8, df[40001:])
c

[2003, 2, 2561, 1350, 8379, 6646, 4361, 7478]


(array([[  4.00000000e+00,   1.00000000e+00,   3.30000000e+01, ...,
          -1.07097000e-01,   1.51220784e-01,  -1.68913871e-01],
        [  6.00000000e+00,   4.00000000e+00,   5.50000000e+01, ...,
          -2.35483274e-01,   5.27072884e-02,  -9.10292044e-02],
        [  5.00000000e+00,   5.00000000e+00,   2.10000000e+01, ...,
          -1.76274970e-01,  -3.78058329e-02,  -1.67819746e-02],
        ..., 
        [  2.00000000e+00,   0.00000000e+00,   5.20000000e+01, ...,
          -2.30151772e-01,   4.94612083e-02,   1.16427943e-01],
        [  6.00000000e+00,   1.00000000e+00,   1.50000000e+01, ...,
          -2.55189776e-01,   5.34484908e-02,  -1.04731187e-01],
        [  5.00000000e+00,   2.10000000e+01,   1.60000000e+01, ...,
           1.70304045e-01,   5.61001971e-02,   4.88971025e-02]], dtype=float32),
 array([[  658.],
        [ 1043.],
        [  410.],
        [  839.],
        [ 1066.],
        [  306.],
        [  548.],
        [  781.]], dtype=float32))

In [207]:
final_embeddings[1]
# res = get_batch(1)
# res

array([ 0.02547936,  0.06881814, -0.13084333,  0.07637364, -0.28064844,
        0.14060961,  0.3406893 , -0.27336642, -0.05460892, -0.27626812,
        0.13503882, -0.1147967 ,  0.10937351, -0.00769361, -0.01512747,
       -0.17756939,  0.02864363, -0.11557664,  0.16492955,  0.06551033,
        0.07619753, -0.09050056, -0.01370888,  0.09181194, -0.14904082,
        0.03825232, -0.04958865,  0.10297409, -0.00137158,  0.03959556,
        0.09600077, -0.02596594,  0.01649722,  0.03757076,  0.01694736,
       -0.00883545,  0.13651466,  0.03001579,  0.1233064 ,  0.17502359,
       -0.04294346,  0.04611364, -0.11848481, -0.16248722,  0.0678286 ,
       -0.10684171, -0.16255438, -0.01875323, -0.01268179, -0.05347537,
       -0.23253202,  0.29666075, -0.00372429, -0.13241382, -0.03298939,
       -0.03545724, -0.04498423, -0.0915895 , -0.11773851,  0.18893519,
        0.10895763,  0.07131821,  0.0736727 , -0.0967845 ], dtype=float32)

In [210]:
def input_placeholders():
    with tf.name_scope('input_layer'):
        features = tf.placeholder(tf.float32, shape=[None, INPUT_FEATURES])
        train_labels = tf.placeholder(tf.float32, shape=[None, 1])
    return features, train_labels
def inference(features, hidden1_units, hidden2_units):
    """ Building the tensorflow model """
    with tf.name_scope('hidden1'):
        weights = tf.Variable(
            tf.truncated_normal([INPUT_FEATURES, hidden1_units],
                                stddev=1.0 / math.sqrt(float(INPUT_FEATURES))),
            name='weights')
        biases = tf.Variable(tf.zeros([hidden1_units]),
                             name='biases')
        hidden1 = tf.nn.relu(tf.matmul(features, weights) + biases)
    # Hidden 2
    with tf.name_scope('hidden2'):
        weights = tf.Variable(
            tf.truncated_normal([hidden1_units, hidden2_units],
                                stddev=1.0 / math.sqrt(float(hidden1_units))),
            name='weights')
        biases = tf.Variable(tf.zeros([hidden2_units]),
                             name='biases')
        hidden2 = tf.nn.relu(tf.matmul(hidden1, weights) + biases)
    # Linear
    with tf.name_scope('softmax_linear'):
        weights = tf.Variable(
            tf.truncated_normal([hidden2_units, 1],
                                stddev=1.0 / math.sqrt(float(hidden2_units))),
            name='weights')
        biases = tf.Variable(tf.zeros([1]),
                             name='biases')
        output_linear = tf.matmul(hidden2, weights) + biases
    return output_linear

In [211]:
def get_loss(output_linear, labels, n_samples):
    """Calculates the loss from the output_linear and the labels.
    Args:
      logits: output_linear tensor, float - [batch_size, 1].
      labels: Labels tensor, int32 - [batch_size].
    Returns:
      loss: Loss tensor of type float.
    """
    labels = tf.to_float(labels)
#     cost = tf.reduce_mean(tf.square(output_linear - labels))
    cost = tf.reduce_sum(tf.pow(output_linear-labels, 2))/(2*n_samples)
    return cost


In [212]:

def training(loss, starter_learning_rate, decay_steps):
    """Sets up the training Ops.
    Creates a summarizer to track the loss over time in TensorBoard.
    Creates an optimizer and applies the gradients to all trainable variables.
    The Op returned by this function is what must be passed to the
    `sess.run()` call to cause the model to train.
    Args:
      loss: Loss tensor, from loss().
      learning_rate: The learning rate to use for gradient descent.
    Returns:
      train_op: The Op for training.
    """
    global_step = tf.Variable(0, name='global_step', trainable=False)
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                               decay_steps, 0.96, staircase=True)   
    # Add a scalar summary for the snapshot loss.
    tf.summary.scalar('loss', loss)
    # Create the gradient descent optimizer with the given learning rate.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    # Create a variable to track the global step.
    # Use the optimizer to apply the gradients that minimize the loss
    # (and also increment the global step counter) as a single training step.
    train_op = optimizer.minimize(loss, global_step=global_step)
    return train_op


In [213]:
def get_evaluation(output_linear, labels):

    diff = tf.subtract(output_linear, labels)
    return tf.reduce_mean(diff)

In [245]:
batch_index = 0
num_steps = 10000
input_batch = 16
starter_learning_rate = 0.001
decay_steps = 500
def start_training(num_steps):
#     sess = tf.InteractiveSession()
#     tf.global_variables_initializer().run()
    with tf.Graph().as_default():
        features, labels = input_placeholders()
        linear_inference = inference(features, 32, 32)
        loss = get_loss(linear_inference, labels, training_size)
        train_op = training(loss, starter_learning_rate, decay_steps)
        evaluation = get_evaluation(linear_inference, labels)
        summary = tf.summary.merge_all()
        init = tf.global_variables_initializer()
        saver = tf.train.Saver()
        sess = tf.Session()
        summary_writer = tf.summary.FileWriter("/tmp/rides/", sess.graph)
        sess.run(init)
        training_data = df[:40000]
        test_data = df[40001:]
        for step in xrange(num_steps):
            start_time = time.time()
            batch_data, label_data = get_batch(input_batch, training_data)
            feed_dict = {features:batch_data, labels:label_data}
            _, loss_value = sess.run([train_op, loss],
                               feed_dict=feed_dict)
            duration = time.time() - start_time

          # Write the summaries and print an overview fairly often.
            if step % 500 == 0:
                # Print status to stdout.
                print('Step %d: loss = %.2f (%.3f sec)' % (step, loss_value, duration))
                # Update the events file.
                summary_str = sess.run(summary, feed_dict=feed_dict)
                summary_writer.add_summary(summary_str, step)
                summary_writer.flush()
            if step % 1000 == 0:
                test_batch_data, test_label_data = get_test_batch(3000, training_data)
                feed_dict = {features:test_batch_data, labels:test_label_data}
                results = sess.run(evaluation, feed_dict=feed_dict)
                print 'Mean difference in testing data : %s' % results
start_training(num_steps)

Step 0: loss = 96.06 (0.027 sec)
Mean difference in testing data : -756.119
Step 500: loss = 51.02 (0.023 sec)
Step 1000: loss = 34.06 (0.018 sec)
Mean difference in testing data : -95.85
Step 1500: loss = 55.71 (0.017 sec)
Step 2000: loss = 35.98 (0.018 sec)
Mean difference in testing data : -90.3855
Step 2500: loss = 51.66 (0.017 sec)
Step 3000: loss = 43.68 (0.017 sec)
Mean difference in testing data : -35.2231
Step 3500: loss = 29.95 (0.017 sec)
Step 4000: loss = 53.39 (0.017 sec)
Mean difference in testing data : -86.0132
Step 4500: loss = 30.18 (0.017 sec)
Step 5000: loss = 47.67 (0.018 sec)
Mean difference in testing data : -56.0713
Step 5500: loss = 44.80 (0.018 sec)
Step 6000: loss = 27.81 (0.017 sec)
Mean difference in testing data : -41.7049
Step 6500: loss = 50.51 (0.018 sec)
Step 7000: loss = 28.11 (0.017 sec)
Mean difference in testing data : -36.1306
Step 7500: loss = 45.36 (0.019 sec)
Step 8000: loss = 44.92 (0.018 sec)
Mean difference in testing data : -11.196
Step 850

In [224]:
training_data = df[:10]
training_data.iloc[0]

row_id                                         0
start_lng                               -74.0091
start_lat                                40.7138
end_lng                                 -74.0043
end_lat                                    40.72
start_timestamp                       1420950819
duration                                     112
start_geohash                            dr5regs
end_geohash                              dr5reuz
start_timestamp_obj    2015-01-10 23:33:39-05:00
start_of_day           2015-01-10 00:00:00-05:00
weekday                                        5
hour                                          23
minute                                        33
Name: 0, dtype: object

In [252]:
def assert_geohashes_in_set(df, reverse_dictionary):
    for index, row in df.iterrows():
        missing = set()
        if not reverse_dictionary.get(row["start_geohash"]):
            missing.add(row["start_geohash"])
        if not reverse_dictionary.get(row["end_geohash"]):
            missing.add(row["end_geohash"])
        if len(missing) > 0:
            raise Exception("end_geohash is not in global dataset: %s"% len(missing))

def transform_input(df):
#     df = pd.read_csv("test.csv")
    df['start_geohash'] = df.apply(geohash_encode_start, axis=1)
    print 'end geohash'
    df['end_geohash'] = df.apply(geohash_encode_end, axis=1)    
    df["start_timestamp_obj"] = df.apply(get_local_time, axis=1)
    df.to_csv("test_mod.csv")
    df["start_of_day"] = df.apply(get_start_of_the_day, axis=1)
    df["weekday"] = df.apply(get_weekday, axis=1)
    df["hour"] = df.apply(get_hour, axis=1)
    df["minute"] = df.apply(get_min, axis=1)
    return df

def handle_test_data():
    df = pd.read_csv("test.csv")
    df = transform_input(df)
    assert_geohashes_in_set(df, reverse_dictionary)
    return df

In [259]:
pd.read_csv("train_mod_temp.csv", index_col=0)


,start_lng,start_lat,end_lng,end_lat,start_timestamp,duration,start_geohash,end_geohash
row_id,,,,,,,,
0,-74.009087,40.713818,-74.004326,40.719986,1420950819,112,dr5regs,dr5reuz
1,-73.971176,40.762428,-74.004181,40.742653,1420950819,1159,dr5rusx,dr5rgbz
2,-73.994957,40.745079,-73.999939,40.734650,1421377541,281,dr5ru1q,dr5rspd
3,-73.991127,40.750080,-73.988609,40.734890,1421377542,636,dr5ru63,dr5rsre
4,-73.945511,40.773724,-73.987434,40.755707,1422173586,705,dr5rvqd,dr5ru77
5,-73.992828,40.701485,-73.992905,40.719978,1422173589,509,dr5rs32,dr5rskb
6,-73.955688,40.772469,-73.974380,40.756672,1422173589,441,dr5rvn7,dr5ruet
7,-73.991249,40.765461,-73.987534,40.768410,1420407892,106,dr5rum1,dr5rume
8,-73.870819,40.773701,-73.968697,40.762680,1421359241,1861,dr5rzn8,dr5ruu9
